In [5]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=c791ab0b49840714ec890353beb7a5508e57bbfffffbe136f5bebfdffeb3b991
  Stored in directory: /home/valeria/.cache/pip/wheels/ae/e7/76/7dd44644d065268ab0e1b4fa2e802fa4bb0157717b7d6c6d92
Successfully built fake-useragent


In [25]:
from bs4 import BeautifulSoup
import json
import tqdm 
import time
import random
import requests as req
from fake_useragent import UserAgent

In [26]:
data = {
    "data":[]
}

In [34]:
for page in range(1,16):
    url = f"https://tyumen.hh.ru/search/vacancy?no_magic=true&L_save_area=true&text=python+разработчик&search_field=name&area=113&salary=&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=50&page={page}&hhtmFrom=vacancy_search_list"
    resp = req.get(url, headers={'User-Agent': UserAgent().chrome})

    soup = BeautifulSoup(resp.text, "lxml")
    tags = soup.find_all(attrs={'data-qa':'serp-item__title'})
    for iter in tqdm.tqdm(tags):
        time.sleep(random.randint(1, 4))
        #print(iter.text, iter.attrs['href'])

        url_object = iter.attrs['href']
        resp_object = req.get(url_object, headers={'User-Agent': UserAgent().chrome})

        soup_object = BeautifulSoup(resp_object.text, "lxml")
        tag_price = soup_object.find(attrs={'data-qa':'vacancy-salary'}).text
        #print(iter.text, tag_price)
        try:
            tag_region = soup_object.find(attrs={'data-qa':'vacancy-view-location'}).text
        except:
            tag_region = soup_object.find(attrs={'data-qa':'vacancy-view-link-location'}).find(attrs={'data-qa':'vacancy-view-raw-address'}).text
                    
        tag_experience = soup_object.find(attrs={'data-qa':'vacancy-experience'}).text

        data["data"].append({"Title":iter.text, "work_experience":tag_experience, "salary":tag_price, 'region':tag_region})
        #print(iter.text, tag_price, tag_region)

    with open('data.json', 'w') as file:
        json.dump(data,file, ensure_ascii=False)

100%|██████████| 20/20 [01:20<00:00,  4.03s/it]
0it [00:00, ?it/s]
